In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer

print(tf.__version__)

2.17.1


In [3]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    as_supervised=True,
    with_info=True
)
train_sentence,train_labels=zip(*[(sent.numpy().decode("utf8"),label.numpy()) for sent,label in train_data])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.BUZV7K_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.BUZV7K_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.BUZV7K_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
tokenizer=Tokenizer(num_words=10000, oov_token="<OOV>")

In [5]:
train_sentence[0]


"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [6]:
tokenizer.fit_on_texts(train_sentence)
train_sequences=tokenizer.texts_to_sequences(train_sentence)

In [7]:
len(train_sequences[0])

117

In [8]:
test_sent,test_labels=zip(*[(sent.numpy().decode('utf8'),label.numpy()) for sent,label in test_data])

In [9]:
test_sequences=tokenizer.texts_to_sequences(test_sent)

In [12]:
train_padded=keras.preprocessing.sequence.pad_sequences(train_sequences,maxlen=100,padding='post',truncating='post')
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

In [14]:
import time
batch_sizes=[16,32,64,128,256,512,1024]
need_time=[]
for j, i in enumerate(batch_sizes):
    # Load dataset with the current batch size
    #train_dataset = load_dataset('train.tfrecord', batch_size=i)
    #test_dataset = load_dataset('test.tfrecord', batch_size=i)

    # Start time for the batch size (resetting time for each batch)
    start = time.time()

    # Define and train the model

    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(10000, 128, input_length=120),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model
    model.fit(x=train_padded,y=train_labels, validation_data=(test_padded,test_labels), epochs=5)

    # End time after training
    end = time.time()
    need_time.append(end-start)

    # Print time taken for each batch size
    print(f"For batch {j + 1} (batch size = {i}), it took {end - start:.4f} seconds")




Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.6122 - loss: 0.6426 - val_accuracy: 0.8046 - val_loss: 0.4219
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.8649 - loss: 0.3379 - val_accuracy: 0.8120 - val_loss: 0.4201
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9141 - loss: 0.2283 - val_accuracy: 0.7989 - val_loss: 0.4762
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9497 - loss: 0.1403 - val_accuracy: 0.8002 - val_loss: 0.6329
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.9731 - loss: 0.0810 - val_accuracy: 0.7809 - val_loss: 0.7539
For batch 1 (batch size = 16), it took 98.4656 seconds
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.6759 - loss: 0.5639 - val_accuracy: 0.8119 - val_loss: 0.4101
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.8768 - loss: 0.3093 - val_accuracy: 0.8026 - val_loss: 0.4265
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step

In [15]:
need_time

[98.46555972099304,
 97.95371341705322,
 91.07335829734802,
 93.23119354248047,
 98.47524571418762,
 84.14314818382263,
 90.95767998695374]

Text: b"Overall I was rather impressed with the pilot. The initial first fifteen minutes were worrying, as it did feel the creators were trying to create a science fiction version of The O.C but this fear is rectified when a terrorist incident occurs and from here the show steps into themes and situations that I very rarely see television tackle.<br /><br />BSG dealt with themes such as monotheism, existentialism, reality, death and terrorism but they were primarily subtext, there for the viewer to contemplate on or ignore if they so choose. Here on the other hand these subjects are the focus of the show and I personally found myself evoking such works as Ghost In The Shell and The Matrix as reference points while watching and being surprised by how well the themes were being discussed. I think if you are a fan of the two I just mentioned or other films/television shows, which deal with the subjects I referenced, I think you will find at least something here.<br /><br />In terms of a s

In [ ]:
# def get_imdb_data():
#   # Load the IMDB Reviews dataset
#   (train_data, valid_data) = tfds.load(
#     'imdb_reviews',
#     split=["train[:90%]", "train[90%:]"],
#     as_supervised=True,
#     with_info=False
#   )
#   test_ds=tfds.load('imdb_reviews',split='test',as_supervised=True)

#   train_ds=(train_data.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE))
#   valid_ds=(valid_data.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE))
#   test_ds=(test_ds.batch(32).prefetch(tf.data.AUTOTUNE))

#   return train_ds,valid_ds,test_ds

# train_ds,valid_ds,test_ds=get_imdb_data()
# import string
# import re
# def clean_text(text):
#   text=tf.strings.lower(text)
#   text=tf.strings.regex_replace(text,"<br />"," ")
#   #text=tf.strings.regex_replace(text,"[%s]"%re.escape(string.punctuation),"")
#   text=tf.strings.regex_replace(text,
# 		f"[{re.escape(string.punctuation)}]", "")
#   return text

# clean_text_train=train_ds.map(lambda text,label:(clean_text(text),label))
# clean_text_valid=valid_ds.map(lambda text,label:(clean_text(text),label))
# clean_text_test=test_ds.map(lambda text,label:(clean_text(text),label))


# clean_train_sentence,clean_train_labels=zip(*[(sent.numpy().decode("utf8"),label.numpy()) for sent,label in clean_text_train])
# clean_valid_sentence,clean_valid_labels=zip(*[(sent.numpy().decode("utf8"),label.numpy()) for sent,label in clean_text_valid])
# clean_test_sentence,clean_test_labels=zip(*[(sent.numpy().decode("utf8"),label.numpy()) for sent,label in clean_text_test])


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import string
import re

# Load and split IMDB data
def get_imdb_data():
    (train_data, valid_data) = tfds.load(
        'imdb_reviews',
        split=["train[:90%]", "train[90%:]"],
        as_supervised=True,
        with_info=False
    )
    test_data = tfds.load('imdb_reviews', split='test', as_supervised=True)

    # Shuffle, batch, and prefetch
    train_ds = train_data.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
    valid_ds = valid_data.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
    test_ds = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

    return train_ds, valid_ds, test_ds

# Text cleaning function
def clean_text(text):
    text = tf.strings.lower(text)  # Convert to lowercase
    text = tf.strings.regex_replace(text, "<br />", " ")  # Remove <br /> tags
    text = tf.strings.regex_replace(text, f"[{re.escape(string.punctuation)}]", "")  # Remove punctuation
    return text

# Apply cleaning function to datasets
def preprocess_dataset(dataset):
    return dataset.map(lambda text, label: (clean_text(text), label), num_parallel_calls=tf.data.AUTOTUNE)

# Load datasets
train_ds, valid_ds, test_ds = get_imdb_data()

# Clean datasets
clean_train_ds = preprocess_dataset(train_ds)
clean_valid_ds = preprocess_dataset(valid_ds)
clean_test_ds = preprocess_dataset(test_ds)


import tensorflow_datasets as tfds


clean_train_sentence, clean_train_labels = zip(*[
    (sent, label) for sent, label in clean_train_ds
])
clean_valid_sentence, clean_valid_labels = zip(*[
    (sent, label) for sent, label in clean_valid_ds
])
clean_test_sentence, clean_test_labels = zip(*[
    (sent, label) for sent, label in clean_test_ds
])



In [ ]:
clean_train_sentence

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# import tensorflow as tf
# import tensorflow_datasets as tfds
# import string
# import re

# # Load and split IMDB data
# def get_imdb_data():
#     (train_data, valid_data) = tfds.load(
#         'imdb_reviews',
#         split=["train[:90%]", "train[90%:]"],
#         as_supervised=True,
#         with_info=False
#     )
#     test_data = tfds.load('imdb_reviews', split='test', as_supervised=True)

#     # Shuffle, batch, and prefetch
#     train_ds = train_data.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
#     valid_ds = valid_data.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
#     test_ds = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

#     return train_ds, valid_ds, test_ds

# # Text cleaning function
# def clean_text(text):
#     text = tf.strings.lower(text)  # Convert to lowercase
#     text = tf.strings.regex_replace(text, "<br />", " ")  # Remove <br /> tags
#     text = tf.strings.regex_replace(text, f"[{re.escape(string.punctuation)}]", "")  # Remove punctuation
#     return text

# # Apply cleaning function to datasets
# def preprocess_dataset(dataset):
#     return dataset.map(lambda text, label: (clean_text(text), label), num_parallel_calls=tf.data.AUTOTUNE)

# # Load datasets
# train_ds, valid_ds, test_ds = get_imdb_data()

# # Clean datasets
# clean_train_ds = preprocess_dataset(train_ds)
# clean_valid_ds = preprocess_dataset(valid_ds)
# clean_test_ds = preprocess_dataset(test_ds)

# # Convert to Python objects (sentences and labels)
# def dataset_to_numpy(dataset):
#     sentences = []
#     labels = []
#     for sent, label in dataset:
#         sentences.append(sent.numpy().decode('utf-8'))
#         labels.append(label.numpy())
#     return sentences, labels

# # Convert each dataset
# clean_train_sentence, clean_train_labels = dataset_to_numpy(clean_train_ds)
# clean_valid_sentence, clean_valid_labels = dataset_to_numpy(clean_valid_ds)
# clean_test_sentence, clean_test_labels = dataset_to_numpy(clean_test_ds)

# # Print a sample to verify
# print("Sample cleaned train sentence:", clean_train_sentence[0])
# print("Sample train label:", clean_train_labels[0])


AttributeError: 'numpy.ndarray' object has no attribute 'decode'

In [ ]:
tokenizer.fit_on_texts(train_sentence)
train_sequences=tokenizer.texts_to_sequences(train_sentence)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
for text_batch, label_batch in clean_text_train.take(5):
    print("Text:", text_batch.numpy()[0])
    print("Label:", label_batch.numpy()[0])
    print("\n")

Text: b'i just came back from the montreal premiere of zero dayand im surprised as hell to find a negative comment on the movie basically the blame is about coccio doing an easy and overplayed social messagewell mrimareviewer its an easy and overplayed critic of movies with a social charge  not that i want to expose my life here but i come from a small town with a similar school than these guys go reject  ignorance on the menu thing isi understand how can young kids can be driven to do such horror high schools have became battle fields of conformity its a real ugly sight you need to fight your way into being like the others its hard to explain bit a lot of people dosent realize that high schools are becoming cemeteries of human intelligence meanwhile parents are closing their eyes and smiling about how their life in their comfortable suburb is perfect  the real motive of the movie isnt about what is driving them its about this deathlike calm suburb and everybody closing their eyes and 

In [ ]:
import tensorflow as tf

class TensorflowRNNModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.rnn = tf.keras.layers.LSTM(rnn_units,
                                        return_sequences=True,
                                        return_state=True)
        self.rnn2 = tf.keras.layers.LSTM(rnn_units,
                                         return_sequences=True,
                                         return_state=True)
        self.dense = tf.keras.layers.Dense(2)

    def call(self, inputs, training=False, mask=None):
        # Pass inputs through the embedding layer
        x = self.embedding(inputs)
        # Pass through the first LSTM
        output, *state = self.rnn(x)
        # Pass through the second LSTM
        output, *state = self.rnn2(output)
        # Pass through the dense layer
        x = self.dense(output)
        return x, state


# Instantiate the model
vocab_size = 1000  # Vocabulary size
embedding_dim = 100  # Embedding dimensions
rnn_units = 64  # Number of LSTM units

model = TensorflowRNNModel(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
#!pip install --upgrade see_rnn


In [ ]:
history=model.fit(clean_text_train,validation_data=clean_text_valid,epochs=10)

Epoch 1/10


ValueError: Exception encountered when calling TensorflowRNNModel.call().

[1mInput 0 of layer "lstm_6" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)[0m

Arguments received by TensorflowRNNModel.call():
  • inputs=tf.Tensor(shape=(None,), dtype=string)
  • training=True
  • mask=None